In [ ]:
from astropop.astrometry import AstrometrySolver, solve_astrometry_framedata, solve_astrometry_xy
from astropop.photometry import background, starfind
from astropop.framedata import FrameData, read_framedata

# get a sample image
from astropy.utils.data import get_pkg_data_filename

In [ ]:
image_name = '/home/julio/J151335.43-372546.2_I_0151.fits'
image_ra = '15:13:35'
image_dec = '-37:25:46'
search_radius = '1 deg'
indexes_path = '/home/julio/Games/INDEXES/5200'

# AstrometrySolver class

In [ ]:
a = AstrometrySolver()

In [ ]:
solution = a.solve_field(image_name,
                         options={'add_path': indexes_path,
                                  'ra': image_ra, 'dec': image_dec, 'radius': search_radius},
                         stdout_loglevel='INFO')

In [ ]:
print(solution.wcs)

# solve_astrometry_xy

In [ ]:
frame = read_framedata(image_name)
bkg, rms = background(frame.data)
sources = starfind(frame.data, background=bkg, noise=rms, threshold=20)
width, height = frame.shape

In [ ]:
sol = solve_astrometry_xy(sources['x'], sources['y'], sources['flux'],
                          width=width, height=height,
                          options={'add_path': indexes_path,
                                  'ra': image_ra, 'dec': image_dec, 'radius': search_radius,
                                  'crpix-center': None, 'tweak-order': 4},
                          stdout_loglevel='INFO')

In [ ]:
sol.wcs

# solve_astrometry_framedata

In [ ]:
sol = solve_astrometry_framedata(frame,
                                 options={'add_path': indexes_path,
                                          'ra': image_ra, 'dec': image_dec, 'radius': search_radius,
                                          'crpix-center': None, 'tweak-order': 4},
                                 stdout_loglevel='INFO')

In [ ]:
sol.wcs

# AstrometricSolution

In [ ]:
sol.wcs

In [ ]:
sol.header

In [ ]:
sol.correspondences